In [1]:
from IPython.display import Image
Image('../input/global-wheat-detection/train/00333207f.jpg',width=600)

In [2]:
from matplotlib import pyplot as plt
# Set default figure size
plt.rcParams['figure.figsize'] = (10.0, 10.0)
import cv2
import pandas as pd
import os
import ast
import numpy as np
import torch

In [3]:
pd.read_csv('../input/global-wheat-detection/train.csv')

In [4]:
#image_loader_function
#load from train.csv
train_dir = '../input/global-wheat-detection/train'
def image_loader(image_id):
    path = os.path.join(train_dir,image_id+'.jpg')
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = image/255 # Scale to [0,1]
    return image



In [39]:
print(image_loader('5e0747034').shape)
image_loader('5e0747034')

In [ ]:
print(torch.from_numpy(image_loader('b6ab77fd7')).permute(2,0,1).shape)
torch.from_numpy(image_loader('b6ab77fd7')).permute(2,0,1)

In [ ]:
torch.from_numpy(image_loader('b6ab77fd7')).permute(2,0,1)[0]

In [ ]:
plt.imshow(image_loader('b6ab77fd7'))

In [5]:
train_df = pd.read_csv('../input/global-wheat-detection/train.csv')
train_df

In [6]:
train_df['bbox'] = train_df['bbox'].apply(ast.literal_eval)
xywh_df = pd.DataFrame(train_df['bbox'].to_list(), columns=["x", "y", "w", "h"])
x2_df = pd.DataFrame(xywh_df.x+xywh_df.w,columns=['x2'])
y2_df = pd.DataFrame(xywh_df.y+xywh_df.h,columns=['y2'])
train_df = train_df.join([xywh_df, x2_df, y2_df])

In [7]:
train_df

In [ ]:
train_df[train_df['image_id']=='b6ab77fd7'][['x', 'y', 'x2', 'y2']].values.shape

In [8]:
#draw boxes on img
def bboxes_on_imgs(image,boxes,color=(255,0,0),acc='1'):
    for box in boxes:
        cv2.rectangle(image,
                      (int(box[0]),int(box[1])),
                       (int(box[2]),int(box[3])),
                       color,3)
        cv2.putText(image, acc, (int(box[0]),int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    return image

In [9]:
# Sample a random training instance and draw the labelled bounding boxes
sample_image_id =  train_df.image_id.sample().item()
#sample_image_id = 'b6ab77fd7'
sample_bboxes = train_df[train_df['image_id']==sample_image_id][['x','y','x2','y2']]


print('image_id:',sample_image_id)
print(len(sample_bboxes.to_numpy()),'bboxes')
plt.imshow(bboxes_on_imgs(image_loader(sample_image_id),sample_bboxes.to_numpy()))

#  **Load Pretained Model** (Torch)

In [10]:
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader, Dataset

In [11]:
# Download a pre-trained bounding box detector
model = fasterrcnn_resnet50_fpn(pretrained=True)

In [12]:
model

In [13]:
model.roi_heads

In [14]:
# Replace the pre-trained bounding box detector head with
# a new one that predicts our desired 2 classes {BACKGROUND, WHEAT}
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_channels=in_features, num_classes=2)

# Verify the model architecture
model.roi_heads

In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [16]:
!nvidia-smi

In [17]:
unique_img_ids = train_df['image_id'].unique()
n_val = int(len(unique_img_ids)*0.2)
val_ids=unique_img_ids[-n_val:]
train_ids=unique_img_ids[:-n_val]

val_df = train_df[train_df['image_id'].isin(val_ids)]
train_df = train_df[train_df['image_id'].isin(train_ids)]

print('validation sample:',len(val_ids))
print('training sample:',len(train_ids))

In [18]:
# Inherit from pytorch Dataset for convenience

class WheatDataset(Dataset):
    def __init__(self, dataframe):
        super().__init__()
        self.df = dataframe
        self.image_ids = dataframe['image_id'].unique()
        
    def __len__(self) -> int:
        return len(self.image_ids)
    
    def __getitem__(self, index: int):
        image_id = self.image_ids[index]
        image = image_loader(image_id).astype(np.float32)
        # Convert the shape from [h,w,c] to [c,h,w] as expected by pytorch
        image = torch.from_numpy(image).permute(2,0,1)
        boxes = torch.as_tensor(self.df[self.df['image_id'] == image_id][['x', 'y', 'x2', 'y2']].values, dtype=torch.float32)
        n_boxes = boxes.shape[0]
        # there is only one foreground class, WHEAT
        labels = torch.ones(n_boxes, dtype=torch.int64)
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        
        return image, target

In [ ]:
torch.ones((10), dtype=torch.int64)

In [ ]:
x = torch.randn(1, 2, 3)
x

In [ ]:
torch.randn(3, 2, 1)

In [ ]:
x.permute(2,1,0)

In [19]:
#Create Pytorch DataLoaders for training and validation datasets

train_dataset = WheatDataset(train_df)
valid_dataset = WheatDataset(val_df)

# A function to bring images with different
# number of bounding boxes into the same batch
def collate_fn(batch):
    return tuple(zip(*batch))

is_training_on_cpu = device == torch.device('cpu')
batch_size = 4 if is_training_on_cpu else 16

train_data_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

Explaination for collate_fn():
https://python.plainenglish.io/understanding-collate-fn-in-pytorch-f9d1742647d3

In [20]:
# Test the data loader
batch_of_images, batch_of_targets = next(iter(train_data_loader))

sample_boxes = batch_of_targets[0]['boxes'].numpy().astype(np.int32)
sample_image = batch_of_images[0].permute(1,2,0).numpy() # convert back from pytorch format

plt.imshow(bboxes_on_imgs(sample_image,sample_boxes,color=(0,200,200)))

# Training the model

In [21]:
# optimiser: stochastic gradient descent
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)

#eash number of epochs
num_epochs = 1

# Prepare the pretained model
model = model.to(device)
model.train()

print('-------------------------------Pretrained FasterRCNN ready-------------------------------')

In [22]:
def move_batch_to_device(images, targets):
    images = list(image.to(device) for image in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    return images, targets

In [23]:
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1,num_epochs))
    average_loss = 0
    for batch_id, (images, targets) in enumerate(train_data_loader):
        # Prepare the batch data
        images, targets = move_batch_to_device(images, targets)

        # Calculate losses
        loss_dict = model(images, targets)
        batch_loss = sum(loss for loss in loss_dict.values()) / len(loss_dict)
        
        # Refresh accumulated optimiser state and minimise losses
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
        # Record stats
        loss_value = batch_loss.item()
        average_loss = average_loss + (loss_value - average_loss) / (batch_id + 1)
        print("Mini-batch: %i/%i Loss: %.4f" % ( batch_id + 1, len(train_data_loader), average_loss), end='\r')
        if batch_id % 100 == 0:
            print("Mini-batch: %i/%i Loss: %.4f" % ( batch_id + 1, len(train_data_loader), average_loss))

In [30]:
model.eval()
cpu_device = torch.device("cpu")
outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]

In [55]:
batch_of_images, batch_of_targets = next(iter(valid_data_loader))

sample_boxes = batch_of_targets[0]['boxes'].numpy().astype(np.int32)
sample_image = batch_of_images[0].permute(1,2,0).numpy() # convert back from pytorch format

plt.imshow(bboxes_on_imgs(sample_image,sample_boxes,color=(0,200,200)))


In [60]:
outputs

In [ ]:
images, targets = next(iter(valid_data_loader))
images, targets = move_batch_to_device(images, targets)
cpu_device = torch.device("cpu")
outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
outputs

In [69]:
for image, output, target in zip(images, outputs, targets): 
    print(output['scores'].cpu().detach().numpy())
    print(output['boxes'].cpu().detach().numpy().astype(np.int32))
    break

In [24]:
# Prepare the model for inference
model.eval()



# Prepare a validation results generator
def make_validation_iter():
    valid_data_iter = iter(valid_data_loader)
    for images, targets in valid_data_iter:
        images, targets = move_batch_to_device(images, targets)

        cpu_device = torch.device("cpu")
        outputs = model(images)
        outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
        for image, output, target in zip(images, outputs, targets): 
            scores = output['scores'].cpu().detach().numpy()
            predicted_boxes = output['boxes'].cpu().detach().numpy().astype(np.int32)
            ground_truth_boxes = target['boxes'].cpu().numpy().astype(np.int32)
            s,p,g = scores,predicted_boxes,ground_truth_boxes
            image = image.permute(1,2,0).cpu().numpy()
            
            
            yield image, ground_truth_boxes, predicted_boxes,scores

validation_iter = make_validation_iter()

In [ ]:
# !pip install GPUtil

# import torch
# from GPUtil import showUtilization as gpu_usage
# from numba import cuda

# def free_gpu_cache():
#     print("Initial GPU Usage")
#     gpu_usage()                             

#     torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

#     print("GPU Usage after emptying the cache")
#     gpu_usage()

# free_gpu_cache()                           

In [ ]:
cpu_device = torch.device("cpu")
outputs = model(images)
outputs = [{k: v.to(cpu_device) for k, v in t.items()} for t in outputs]
for image, output, target in zip(images, outputs, targets): 
    scores = output['scores'].cpu().detach().numpy().astype(np.int32)
    predicted_boxes = output['boxes'].cpu().detach().numpy().astype(np.int32)
    ground_truth_boxes = target['boxes'].cpu().numpy().astype(np.int32)
    image = image.permute(1,2,0).cpu().numpy()
    break
print(scores)

In [25]:
#draw boxes on img
def bboxes_on_imgs_prob(image,boxes,scores,color=(255,0,0)):
    for box,score in zip(boxes,scores):
        cv2.rectangle(image,
                      (int(box[0]),int(box[1])),
                       (int(box[2]),int(box[3])),
                       color,3)
        cv2.putText(image, str(score), (int(box[0]),int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
    return image

In [ ]:
scores

In [26]:
image, ground_truth_boxes, predicted_boxes,scores = next(validation_iter)
image = bboxes_on_imgs_prob(image,predicted_boxes,scores,(255,0,0))
image = bboxes_on_imgs_prob(image,ground_truth_boxes,scores,(0,255,0))
plt.imshow(image)